# ML with Spark

**Christian Alis**

Apache Spark has its own ML library, MLlib. Although it has both an RDD- and DataFrames-based API, the former is already deprecated so you should use only the latter. The DataFrames-based API follows a workflow similar to scikit-learn.

## Basic statistics

To introduce MLlib, let us try performing a simple task which is taking the Spearman correlation of vectors.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

22/03/06 15:33:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/06 15:33:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/06 15:33:39 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation

data = [(Vectors.sparse(4, [(0, 1.0), (3, -2.0)]),), # [1, 0, 0, -2]
        (Vectors.dense([4.0, 5.0, 0.0, 3.0]),),
        (Vectors.sparse(4, [(0, 9.0), (3, 1.0)]),)] # [9, 0, 0, 1]

display(data)

df = spark.createDataFrame(data, ["features"])
display(df.toPandas())
print(Correlation.corr(df, "features", "spearman").collect()[0][0])

[(SparseVector(4, {0: 1.0, 3: -2.0}),),
 (DenseVector([4.0, 5.0, 0.0, 3.0]),),
 (SparseVector(4, {0: 9.0, 3: 1.0}),)]

,features
0,"(1.0, 0.0, 0.0, -2.0)"
1,"[4.0, 5.0, 0.0, 3.0]"
2,"(9.0, 0.0, 0.0, 1.0)"


DenseMatrix([[1.       , 0.       ,       nan, 0.5      ],
             [0.       , 1.       ,       nan, 0.8660254],
             [      nan,       nan, 1.       ,       nan],
             [0.5      , 0.8660254,       nan, 1.       ]])


22/03/06 16:05:25 WARN PearsonCorrelation: Pearson correlation matrix contains NaN values.


Notice that each row can be a sparse or dense vector, and that all rows need not be of the same kind of vector. Correlation is done on the matrix formed by the dataframe column `features`. Thus, correlation is performed on the matrix:

$$
\left(
\begin{array}{cccc}
1 & 0 & 0 & -2 \\
4 & 5 & 0 & 3 \\
9 & 0 & 0 & 1
\end{array}
\right)
$$

The vectors are the columns of this matrix, thus, the resulting correlation matrix is $4 \times 4$.

**Exercise 1**

Create a function `corr` that returns the Spearman correlation of two Spark dataframe columns.

In [3]:
from pyspark.ml.feature import VectorAssembler
def corr(df, col1, col2):  
    df = VectorAssembler(inputCols=[col1, col2],
                        outputCol='features').transform(df)
    return Correlation.corr(df, 'features', 'spearman').collect()[0][0][0,1]

In [11]:
from pyspark.ml.feature import VectorAssembler
import pandas as pd
df_corr = spark.createDataFrame(
    pd.DataFrame([[2, 2, 6],
                  [5, 4, 8],
                  [8, 3, 1],
                  [6, 3, 8]],
                 columns=['col1', 'col2', 'col3'])
)
VectorAssembler(inputCols=['col1', 'col2'], outputCol='features').transform(df_corr).toPandas()

,col1,col2,col3,features
0,2,2,6,"[2.0, 2.0]"
1,5,4,8,"[5.0, 4.0]"
2,8,3,1,"[8.0, 3.0]"
3,6,3,8,"[6.0, 3.0]"


In [12]:
Correlation.corr?

In [4]:
import pandas as pd
from numpy.testing import assert_almost_equal, assert_array_equal
df_corr = spark.createDataFrame(
    pd.DataFrame([[2, 2, 6],
                  [5, 4, 8],
                  [8, 3, 1],
                  [6, 3, 8]],
                 columns=['col1', 'col2', 'col3'])
)
assert_almost_equal(corr(df_corr, 'col1', 'col2'), 
                    0.3162277660168371)

## Transformers and Estimators

Apache Spark also has the notion of transformers and estimators. As in scikit-learn, transformers accept a dataframe and returns a dataframe. On the other hand, unlike in scikit-learn, estimators accept a dataframe and returns a transformer. In general, the `fit` function does not return the same transformer or estimator but a trained model or transformer. You will then use its `transform` method to transform an input dataset or make predictions. There is no `predict` or `fit_predict` methods. 

Prediction is done using a `transform` call. It will return a dataframe with an additional column holding the predictions. Typical parameters of transformers and estimators are `inputCol` or `inputCols` and `outputCol`. If it's an `inputCol` then it usually expects that `inputCol` to contain a `Vector` corresponding to the different column values for that row.

## Feature Engineering

Let us look at how to do feature engineering on SparkML. As an example, let's convert `PULocationID` into an integer.

In [13]:
df_taxi = (spark.read.csv('/mnt/data/public/nyctaxi/all/'
                          'yellow_tripdata_2017-12.csv',
                          header=True)
                .limit(10000)
                .persist())

In [14]:
df_taxi.limit(10).toPandas()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2017-12-01 00:12:00,2017-12-01 00:12:51,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8
1,1,2017-12-01 00:13:37,2017-12-01 00:13:47,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8
2,1,2017-12-01 00:14:15,2017-12-01 00:15:05,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8
3,1,2017-12-01 00:15:33,2017-12-01 00:15:37,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8
4,1,2017-12-01 00:50:03,2017-12-01 00:53:35,1,.00,1,N,145,145,2,4,0.5,0.5,0,0,0.3,5.3
5,1,2017-12-01 00:14:20,2017-12-01 00:28:35,1,4.20,1,N,82,258,2,15,0.5,0.5,0,0,0.3,16.3
6,1,2017-12-01 00:20:32,2017-12-01 00:31:24,1,5.40,1,N,50,116,2,17,0.5,0.5,0,0,0.3,18.3
7,1,2017-12-01 00:01:46,2017-12-01 00:12:19,1,1.90,1,N,161,107,1,9,0.5,0.5,2.05,0,0.3,12.35
8,1,2017-12-01 00:17:52,2017-12-01 00:32:35,1,3.30,1,N,107,263,1,12.5,0.5,0.5,2.07,0,0.3,15.87
9,1,2017-12-01 00:10:00,2017-12-01 00:24:35,1,2.80,1,N,264,87,1,12,0.5,0.5,2.65,0,0.3,15.95


One way of doing so is by casting the string column into a double which is okay in this case since `PULocationID` is numeric.

In [15]:
(df_taxi
 .withColumn('PULocationID_index', df_taxi.PULocationID.astype('double'))
 .limit(10)
 .toPandas())

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PULocationID_index
0,1,2017-12-01 00:12:00,2017-12-01 00:12:51,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8,226.0
1,1,2017-12-01 00:13:37,2017-12-01 00:13:47,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8,226.0
2,1,2017-12-01 00:14:15,2017-12-01 00:15:05,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8,226.0
3,1,2017-12-01 00:15:33,2017-12-01 00:15:37,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8,226.0
4,1,2017-12-01 00:50:03,2017-12-01 00:53:35,1,.00,1,N,145,145,2,4,0.5,0.5,0,0,0.3,5.3,145.0
5,1,2017-12-01 00:14:20,2017-12-01 00:28:35,1,4.20,1,N,82,258,2,15,0.5,0.5,0,0,0.3,16.3,82.0
6,1,2017-12-01 00:20:32,2017-12-01 00:31:24,1,5.40,1,N,50,116,2,17,0.5,0.5,0,0,0.3,18.3,50.0
7,1,2017-12-01 00:01:46,2017-12-01 00:12:19,1,1.90,1,N,161,107,1,9,0.5,0.5,2.05,0,0.3,12.35,161.0
8,1,2017-12-01 00:17:52,2017-12-01 00:32:35,1,3.30,1,N,107,263,1,12.5,0.5,0.5,2.07,0,0.3,15.87,107.0
9,1,2017-12-01 00:10:00,2017-12-01 00:24:35,1,2.80,1,N,264,87,1,12,0.5,0.5,2.65,0,0.3,15.95,264.0


But to convert a string column, which could be a label, into an integer, we could use `StringIndexer` instead.

In [16]:
from pyspark.ml.feature import StringIndexer
si_pu = StringIndexer(inputCol='PULocationID', outputCol='PULocationID_index')
si_pu_trained = si_pu.fit(df_taxi)

Notice that there's no such thing as `si_pu.fit_transform` or `si_pu.fit_predict`. The `fit` method returns a model which is a different class from `si_pu`.

In [17]:
type(si_pu)

pyspark.ml.feature.StringIndexer

In [18]:
type(si_pu_trained)

pyspark.ml.feature.StringIndexerModel

In [23]:
si_pu_trained.labelsArray[0][58] # if you want to reverse the transformation, just index from here

'226'

We use the model to transform the `inputCol` in the input dataset into a new column `outputCol` in the same dataset.

In [19]:
df_taxi = si_pu_trained.transform(df_taxi)

In [20]:
(df_taxi
 .limit(10)
 .toPandas())

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PULocationID_index
0,1,2017-12-01 00:12:00,2017-12-01 00:12:51,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8,58.0
1,1,2017-12-01 00:13:37,2017-12-01 00:13:47,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8,58.0
2,1,2017-12-01 00:14:15,2017-12-01 00:15:05,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8,58.0
3,1,2017-12-01 00:15:33,2017-12-01 00:15:37,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8,58.0
4,1,2017-12-01 00:50:03,2017-12-01 00:53:35,1,.00,1,N,145,145,2,4,0.5,0.5,0,0,0.3,5.3,57.0
5,1,2017-12-01 00:14:20,2017-12-01 00:28:35,1,4.20,1,N,82,258,2,15,0.5,0.5,0,0,0.3,16.3,86.0
6,1,2017-12-01 00:20:32,2017-12-01 00:31:24,1,5.40,1,N,50,116,2,17,0.5,0.5,0,0,0.3,18.3,27.0
7,1,2017-12-01 00:01:46,2017-12-01 00:12:19,1,1.90,1,N,161,107,1,9,0.5,0.5,2.05,0,0.3,12.35,4.0
8,1,2017-12-01 00:17:52,2017-12-01 00:32:35,1,3.30,1,N,107,263,1,12.5,0.5,0.5,2.07,0,0.3,15.87,10.0
9,1,2017-12-01 00:10:00,2017-12-01 00:24:35,1,2.80,1,N,264,87,1,12,0.5,0.5,2.65,0,0.3,15.95,26.0


We can perform one-hot encoding using `OneHotEncoder` but it only accepts categorical indices not strings directly. Hence, the need for `StringIndexer` and not just conversion to a `double` type.

In [12]:
from pyspark.ml.feature import OneHotEncoder
ohe_pu = OneHotEncoder(
    inputCol='PULocationID_index', outputCol='PULocationID_ohe')
ohe_pu_trained = ohe_pu.fit(df_taxi)

In [13]:
(ohe_pu_trained
 .transform(df_taxi)
 .limit(10)
 .toPandas())

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PULocationID_index,PULocationID_ohe
0,1,2017-12-01 00:12:00,2017-12-01 00:12:51,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8,58.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,2017-12-01 00:13:37,2017-12-01 00:13:47,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8,58.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1,2017-12-01 00:14:15,2017-12-01 00:15:05,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8,58.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1,2017-12-01 00:15:33,2017-12-01 00:15:37,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8,58.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1,2017-12-01 00:50:03,2017-12-01 00:53:35,1,.00,1,N,145,145,2,4,0.5,0.5,0,0,0.3,5.3,57.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,1,2017-12-01 00:14:20,2017-12-01 00:28:35,1,4.20,1,N,82,258,2,15,0.5,0.5,0,0,0.3,16.3,86.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,1,2017-12-01 00:20:32,2017-12-01 00:31:24,1,5.40,1,N,50,116,2,17,0.5,0.5,0,0,0.3,18.3,27.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,1,2017-12-01 00:01:46,2017-12-01 00:12:19,1,1.90,1,N,161,107,1,9,0.5,0.5,2.05,0,0.3,12.35,4.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
8,1,2017-12-01 00:17:52,2017-12-01 00:32:35,1,3.30,1,N,107,263,1,12.5,0.5,0.5,2.07,0,0.3,15.87,10.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,1,2017-12-01 00:10:00,2017-12-01 00:24:35,1,2.80,1,N,264,87,1,12,0.5,0.5,2.65,0,0.3,15.95,26.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


`OneHotIndexer` generates a sparse vector.

In [14]:
ohe_pu_trained.transform(df_taxi).printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- PULocationID_index: double (nullable = false)
 |-- PULocationID_ohe: vector (nullable = true)



For many SparkML models, the input is just a single column of vector. To create a vector from several columns of numeric variables, we can use `VectorAssembler`.

In [15]:
df_taxi = (df_taxi
           .withColumn('PULocationID', df_taxi.PULocationID.astype('double'))
           .withColumn('DOLocationID', df_taxi.DOLocationID.astype('double')))

In [16]:
from pyspark.ml.feature import VectorAssembler
(VectorAssembler(inputCols=['PULocationID', 'DOLocationID'],
                     outputCol='features')
.transform(df_taxi)
.printSchema())

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: double (nullable = true)
 |-- DOLocationID: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- PULocationID_index: double (nullable = false)
 |-- features: vector (nullable = true)



In [17]:
(VectorAssembler(inputCols=['PULocationID', 'DOLocationID'],
                     outputCol='features')
 .transform(df_taxi)
 .limit(10)
 .toPandas())

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PULocationID_index,features
0,1,2017-12-01 00:12:00,2017-12-01 00:12:51,1,.00,1,N,226.0,226.0,3,2.5,0.5,0.5,0,0,0.3,3.8,58.0,"[226.0, 226.0]"
1,1,2017-12-01 00:13:37,2017-12-01 00:13:47,1,.00,1,N,226.0,226.0,3,2.5,0.5,0.5,0,0,0.3,3.8,58.0,"[226.0, 226.0]"
2,1,2017-12-01 00:14:15,2017-12-01 00:15:05,1,.00,1,N,226.0,226.0,3,2.5,0.5,0.5,0,0,0.3,3.8,58.0,"[226.0, 226.0]"
3,1,2017-12-01 00:15:33,2017-12-01 00:15:37,1,.00,1,N,226.0,226.0,3,2.5,0.5,0.5,0,0,0.3,3.8,58.0,"[226.0, 226.0]"
4,1,2017-12-01 00:50:03,2017-12-01 00:53:35,1,.00,1,N,145.0,145.0,2,4,0.5,0.5,0,0,0.3,5.3,57.0,"[145.0, 145.0]"
5,1,2017-12-01 00:14:20,2017-12-01 00:28:35,1,4.20,1,N,82.0,258.0,2,15,0.5,0.5,0,0,0.3,16.3,86.0,"[82.0, 258.0]"
6,1,2017-12-01 00:20:32,2017-12-01 00:31:24,1,5.40,1,N,50.0,116.0,2,17,0.5,0.5,0,0,0.3,18.3,27.0,"[50.0, 116.0]"
7,1,2017-12-01 00:01:46,2017-12-01 00:12:19,1,1.90,1,N,161.0,107.0,1,9,0.5,0.5,2.05,0,0.3,12.35,4.0,"[161.0, 107.0]"
8,1,2017-12-01 00:17:52,2017-12-01 00:32:35,1,3.30,1,N,107.0,263.0,1,12.5,0.5,0.5,2.07,0,0.3,15.87,10.0,"[107.0, 263.0]"
9,1,2017-12-01 00:10:00,2017-12-01 00:24:35,1,2.80,1,N,264.0,87.0,1,12,0.5,0.5,2.65,0,0.3,15.95,26.0,"[264.0, 87.0]"


Individual elements of vectors can be converted into categorical indices using `VectorIndexer`. A vector element will be considered a categorical variable if the number of unique values for that element does not exceed `maxCategories`.

In [18]:
df_taxi = (VectorAssembler(inputCols=['PULocationID', 'DOLocationID'],
                     outputCol='features')
 .transform(df_taxi))

In [19]:
from pyspark.ml.feature import VectorIndexer
vi = VectorIndexer(maxCategories=1000, 
                   inputCol='features', 
                   outputCol='features_indexed')
vi_trained = vi.fit(df_taxi)
df_taxi = vi_trained.transform(df_taxi)

In [20]:
df_taxi.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: double (nullable = true)
 |-- DOLocationID: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- PULocationID_index: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- features_indexed: vector (nullable = true)



In [21]:
df_taxi.limit(10).toPandas()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PULocationID_index,features,features_indexed
0,1,2017-12-01 00:12:00,2017-12-01 00:12:51,1,.00,1,N,226.0,226.0,3,2.5,0.5,0.5,0,0,0.3,3.8,58.0,"[226.0, 226.0]","[96.0, 172.0]"
1,1,2017-12-01 00:13:37,2017-12-01 00:13:47,1,.00,1,N,226.0,226.0,3,2.5,0.5,0.5,0,0,0.3,3.8,58.0,"[226.0, 226.0]","[96.0, 172.0]"
2,1,2017-12-01 00:14:15,2017-12-01 00:15:05,1,.00,1,N,226.0,226.0,3,2.5,0.5,0.5,0,0,0.3,3.8,58.0,"[226.0, 226.0]","[96.0, 172.0]"
3,1,2017-12-01 00:15:33,2017-12-01 00:15:37,1,.00,1,N,226.0,226.0,3,2.5,0.5,0.5,0,0,0.3,3.8,58.0,"[226.0, 226.0]","[96.0, 172.0]"
4,1,2017-12-01 00:50:03,2017-12-01 00:53:35,1,.00,1,N,145.0,145.0,2,4,0.5,0.5,0,0,0.3,5.3,57.0,"[145.0, 145.0]","[60.0, 107.0]"
5,1,2017-12-01 00:14:20,2017-12-01 00:28:35,1,4.20,1,N,82.0,258.0,2,15,0.5,0.5,0,0,0.3,16.3,86.0,"[82.0, 258.0]","[33.0, 202.0]"
6,1,2017-12-01 00:20:32,2017-12-01 00:31:24,1,5.40,1,N,50.0,116.0,2,17,0.5,0.5,0,0,0.3,18.3,27.0,"[50.0, 116.0]","[21.0, 82.0]"
7,1,2017-12-01 00:01:46,2017-12-01 00:12:19,1,1.90,1,N,161.0,107.0,1,9,0.5,0.5,2.05,0,0.3,12.35,4.0,"[161.0, 107.0]","[68.0, 77.0]"
8,1,2017-12-01 00:17:52,2017-12-01 00:32:35,1,3.30,1,N,107.0,263.0,1,12.5,0.5,0.5,2.07,0,0.3,15.87,10.0,"[107.0, 263.0]","[43.0, 207.0]"
9,1,2017-12-01 00:10:00,2017-12-01 00:24:35,1,2.80,1,N,264.0,87.0,1,12,0.5,0.5,2.65,0,0.3,15.95,26.0,"[264.0, 87.0]","[118.0, 62.0]"


## Classification and Regression

Just like in scikit-learn, in MLlib, we instantiate an ML model, fit data on it then use the trained data to predict. Data splitting is done using `randomSplit` and prediction is via the `transform` method of the trained model.

In [22]:
from pyspark.ml.feature import (StringIndexer, VectorAssembler, VectorIndexer)
from pyspark.ml.regression import RandomForestRegressor

df_taxi = (df_taxi.withColumn('total_amount', 
                              df_taxi['total_amount'].astype('float'))
                  .cache())

df_training, df_test = df_taxi.randomSplit([0.7, 0.3])

rf = RandomForestRegressor(featuresCol='features_indexed',
                           labelCol='total_amount',
                           maxBins=1000)
rf_trained = rf.fit(df_training)
df_predict = rf_trained.transform(df_test)

df_predict[['features', 'total_amount', 'prediction']].show()

+-------------+------------+------------------+
|     features|total_amount|        prediction|
+-------------+------------+------------------+
|[148.0,148.0]|       24.95|12.131289659025446|
|[140.0,146.0]|       18.48|17.097137981219525|
|[137.0,148.0]|       12.05|11.508767391534121|
|[114.0,145.0]|       28.55|20.136499514465225|
|[141.0,262.0]|        18.8|16.095632671971217|
|[231.0,265.0]|       60.95| 103.2923289092543|
|[138.0,263.0]|       41.91| 52.48693483037897|
|[148.0,137.0]|        16.3|11.843556650570866|
|[186.0,238.0]|        19.2|14.351107686600301|
|[140.0,140.0]|         5.3|12.805309388680332|
|[255.0,263.0]|        30.0| 17.79750434455514|
|[148.0,249.0]|       12.25| 11.89426435610714|
|[144.0,107.0]|        11.6| 11.43229389748015|
| [37.0,255.0]|        14.8|15.523961084168945|
| [145.0,61.0]|       29.15| 27.74391242986461|
| [231.0,88.0]|       12.36| 16.46464018790974|
|[138.0,145.0]|        26.6| 47.34546634035111|
|[114.0,226.0]|        33.5|  21.6052290

**Exercise 2**

Create a function `predict_payment_type` that accepts the training and prediction data in the form of Spark dataframes then returns the prediction as a list. Use a random forest model as the classifier, the categorical variables `PULocationID` and `DOLocationID` as features, and `payment_type` as target. Set the `seed` of the classifier to 2020 and `maxBins` to 1000. Do not perform cross-validation or further split the training data.

In [23]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer

def predict_payment_type(df_training, df_to_predict):
    pipe = Pipeline(stages=[
        StringIndexer(inputCol='PULocationID', outputCol='PULocationID_index'),
        StringIndexer(inputCol='DOLocationID', outputCol='DOLocationID_index'),
        VectorAssembler(inputCols=['PULocationID_index', 'DOLocationID_index'],
                        outputCol='features'),
        VectorIndexer(inputCol='features', outputCol='features_indexed'),
        RandomForestClassifier(featuresCol='features_indexed',
                              labelCol='payment_type_index',
                              maxBins=1000,
                              seed=2020)
    ])

    df_training_ = df_training.withColumn('payment_type_index', 
                               df_training.payment_type.astype('int'))

    model_trained = pipe.fit(df_training_)
    df_predict = model_trained.transform(df_to_predict)
    final_pred = df_predict[['features', 'prediction']].select('prediction')
    return final_pred.toPandas()['prediction'].astype('int').tolist()

In [24]:
df_taxi = (spark.read.csv('/mnt/data/public/nyctaxi/all/'
                          'yellow_tripdata_2017-12.csv',
                          header=True)
                .limit(10000))
df_to_predict = spark.createDataFrame(
    [[87, 102],
     [107, 95],
     [17, 28],
     [82, 258],
     [67, 48]],
    schema=['PULocationID', 'DOLocationID']
)
y_pred = predict_payment_type(df_taxi, df_to_predict)
assert_array_equal(y_pred, [1, 1, 2, 1, 1])


## Pipelines

As you may have noticed, instantiating an estimator or transformer, fitting it to the data then storing its result, then using that to transform the input is quite tedious. In scikit-learn, a way for simplifying this process is to create a `Pipeline`, which Spark also has.

In [25]:
from pyspark.ml import Pipeline

df_taxi = (spark.read.csv('/mnt/data/public/nyctaxi/all/'
                          'yellow_tripdata_2017-12.csv',
                          header=True)
                .limit(10000))
df_taxi = (df_taxi.withColumn('total_amount', 
                             df_taxi['total_amount'].astype('float'))
                  .cache())

df_training, df_test = df_taxi.randomSplit([0.7, 0.3])

pipe = Pipeline(stages=[
    StringIndexer(inputCol='PULocationID', outputCol='PULocationID_index'),
    StringIndexer(inputCol='DOLocationID', outputCol='DOLocationID_index'),
    VectorAssembler(inputCols=['PULocationID_index', 'DOLocationID_index'],
                    outputCol='features'),
    VectorIndexer(inputCol='features', outputCol='features_indexed'),
    RandomForestRegressor(featuresCol='features_indexed',
                          labelCol='total_amount',
                          maxBins=1000)
])

model_trained = pipe.fit(df_training)
df_predict = model_trained.transform(df_test)

df_predict[['features', 'total_amount', 'prediction']].show()

+-----------+------------+------------------+
|   features|total_amount|        prediction|
+-----------+------------+------------------+
| [29.0,1.0]|        10.8| 18.44345605515573|
| [50.0,8.0]|        20.3|12.016707907969652|
| [8.0,60.0]|       28.55|21.332079675954677|
|[26.0,39.0]|        18.8|12.776414819045167|
|[31.0,37.0]|        6.35|11.949968595288466|
|  [2.0,4.0]|         7.3|12.115045882425246|
|[17.0,38.0]|         4.3| 12.09844487781461|
| [28.0,1.0]|         5.8|11.393239950112209|
|  [0.0,6.0]|         5.8|11.097208865396638|
| [16.0,2.0]|        18.8|11.764880285791072|
|[66.0,14.0]|       41.91| 33.81084297122338|
| [8.0,18.0]|       11.15|11.528496989920978|
|  [1.0,0.0]|         6.3|11.476033509723681|
|[34.0,24.0]|        13.3|11.719880743258257|
|  [4.0,3.0]|       12.25| 11.73712438829062|
| [23.0,5.0]|        11.6|11.375205508702189|
| [12.0,9.0]|        10.3|11.318500386868447|
|[14.0,76.0]|       12.36|14.941578443780603|
| [7.0,80.0]|        26.8|27.12376

## Saving and loading models

Both pipeline and trained models can be saved using the `save` method of the model.

In [26]:
!rm -rf example*
pipe.save('example-untrained')
model_trained.save('example-trained')

The pipeline and trained models can then be loaded using `load`.

In [27]:
from pyspark.ml import PipelineModel
pipe2 = Pipeline.load('example-untrained')
model_trained2 = PipelineModel.load('example-trained')

## Clustering

MLlib has a few models for clustering as well.

In [28]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

df_taxi2 = (spark.read.csv('/mnt/data/public/nyctaxi/all/'
                          'yellow_tripdata_2016-01.csv',
                          header=True)
                .limit(10000))
df_taxi2 = (df_taxi2.withColumn('pickup_longitude', 
                                df_taxi2['pickup_longitude'].astype('float'))
                    .withColumn('pickup_latitude', 
                                df_taxi2['pickup_latitude'].astype('float'))
                    .cache())

pipe = Pipeline(stages=[
    VectorAssembler(inputCols=['pickup_longitude', 'pickup_latitude'],
                    outputCol='features'),
    KMeans(k=8)
])

model_trained = pipe.fit(df_taxi2)
df_predict = model_trained.transform(df_taxi2)

evaluator = ClusteringEvaluator()

print('Silhouette score: ', evaluator.evaluate(df_predict))

df_predict[['features', 'prediction']].show()

Silhouette score:  0.5065738153894321
+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[-73.990371704101...|         7|
|[-73.980781555175...|         0|
|[-73.984550476074...|         5|
|[-73.993469238281...|         0|
|[-73.960624694824...|         2|
|[-73.980117797851...|         7|
|[-73.994056701660...|         0|
|[-73.979423522949...|         4|
|[-73.947151184082...|         2|
|[-73.998344421386...|         0|
|[-74.006149291992...|         7|
|[-73.969329833984...|         4|
|[-73.989021301269...|         0|
|[-74.004302978515...|         7|
|[-73.991996765136...|         0|
|[-73.985160827636...|         4|
|[-73.973091125488...|         2|
|[-73.982101440429...|         4|
|[-73.994842529296...|         0|
|[-73.953033447265...|         5|
+--------------------+----------+
only showing top 20 rows



## Hyperparameter tuning

MLlib has support for cross-validation and grid search as well. For example, we can use it to pick the best value fo $k$ based on the silhouette coefficient.

In [29]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid = (ParamGridBuilder()
                .addGrid(pipe.getStages()[-1].k, [2, 4, 8]) #add grid here specificies the variable you want to change
                .build())

cv = CrossValidator(estimator=pipe,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator)
cv_trained = cv.fit(df_taxi2)
print(cv_trained.bestModel.stages[-1].explainParam('k'))

k: The number of clusters to create. Must be > 1. (default: 2, current: 2)


In [24]:
float?

# Frequent itemset mining

Unlike scikit-learn, MLlib has support for frequent itemset mining.

In [30]:
from pyspark.sql.functions import collect_set
from pyspark.ml.fpm import FPGrowth

df_retail = spark.read.csv('/mnt/data/public/retaildata/Online Retail.csv',
                           header=True)
df_trans = (df_retail.groupby('InvoiceNo')
                     .agg(collect_set('StockCode').alias('trans'))
                     .cache())

fpgrowth = FPGrowth(itemsCol="trans", minSupport=0.03, minConfidence=0.1)
fpgrowth_trained = fpgrowth.fit(df_trans)

print('Frequent itemsets:')
fpgrowth_trained.freqItemsets.show()

print('Association rules:')
fpgrowth_trained.associationRules.show()

print('Consequents from association rules:')
fpgrowth_trained.transform(df_trans).show()

22/02/25 23:58:43 WARN FPGrowth: Input data is not cached.                      


Frequent itemsets:


+---------------+----+
|          items|freq|
+---------------+----+
|       [85123A]|2246|
|        [22423]|2172|
|       [85099B]|2135|
|        [47566]|1706|
|        [20725]|1608|
|        [84879]|1468|
|        [22720]|1462|
|        [22197]|1442|
|        [21212]|1334|
|        [22383]|1306|
|        [20727]|1295|
|        [22457]|1266|
|         [POST]|1254|
|        [23203]|1249|
|        [22386]|1231|
|[22386, 85099B]| 833|
|        [22960]|1220|
|        [22469]|1214|
|        [21931]|1201|
|        [22411]|1187|
+---------------+----+
only showing top 20 rows

Association rules:


+----------+----------+------------------+-----------------+-------------------+
|antecedent|consequent|        confidence|             lift|            support|
+----------+----------+------------------+-----------------+-------------------+
|   [22699]|   [22697]|               0.7| 17.1523178807947|0.03027027027027027|
|   [22386]|  [85099B]|0.6766856214459789| 8.20897311262335|0.03216216216216216|
|  [85099B]|   [22386]|0.3901639344262295|8.208973112623351|0.03216216216216216|
|   [22697]|   [22699]|0.7417218543046358| 17.1523178807947|0.03027027027027027|
+----------+----------+------------------+-----------------+-------------------+

Consequents from association rules:
+---------+--------------------+----------+
|InvoiceNo|               trans|prediction|
+---------+--------------------+----------+
|   536596|[22900, 22114, 84...|        []|
|   536938|[22112, 21931, 84...|  [85099B]|
|   537252|             [22197]|        []|
|   537691|[22505, 46000R, 2...|        []|
|   538

## Collaborative filtering

MLlib has support for collaborative filtering using ALS with both implicit and explicit ratings.

In [31]:
from pyspark.ml.recommendation import ALS

df_ratings = (spark.read.csv('/mnt/data/public/movielens/20m/ml-20m/'
                             'ratings.csv',
                             header=True,
                             schema="""
                             userId INT,
                             movieId INT,
                             rating FLOAT,
                             timestamp INT
                             """)
                   .limit(10000)
                   .cache())
df_training, df_test = df_ratings.randomSplit([0.8, 0.2])

als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", 
          ratingCol="rating")
als_trained = als.fit(df_training)

df_predict = als_trained.transform(df_test)

users = df_ratings.select(als.getUserCol()).distinct().limit(3)
als_trained.recommendForUserSubset(users, 10).show()

movies = df_ratings.select(als.getItemCol()).distinct().limit(3)
als_trained.recommendForItemSubset(movies, 10).show()

22/02/25 23:59:08 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/02/25 23:59:08 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{1231, 5.79897},...|
|     3|[{3753, 6.3427973...|
|     2|[{2353, 8.399177}...|
+------+--------------------+



+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     29|[{27, 3.0145586},...|
|     32|[{81, 5.452299}, ...|
|      2|[{52, 6.4459743},...|
+-------+--------------------+

